### Post Import Fixes
> Post fixes that need to be applied after importing litesearch package.

We need to apply some post fix for mac to load the dylib properly for usearch sqlite path to be loaded correctly.


In [1]:
#| default_exp postfix

In [2]:
#| export
import os, subprocess, platform

In [7]:
#| export
def usearch_fix(q=True):
    """Apply usearch macOS fix if on macOS."""
    if not q: print('Applying usearch macOS fix if required...')
    try:
        from usearch import sqlite_path
        dylib = sqlite_path()+'.dylib'
        if not q: print('usearch dylib path: ', dylib)
        if platform.system() != "Darwin":
            if not q: print('Not on macOS, skipping usearch fix.')
            return
        cmd = ['install_name_tool', '-add_rpath', '/usr/lib', dylib]
        r = subprocess.run(cmd, capture_output=True, text=True, check=True)
        if r.returncode == 0:  print(f'✓ Applied usearch fix: Added /usr/lib rpath to {dylib}') if not q else None
        else: print(f'✗ Failed to apply fix: {r.stderr}') if not q else None
    except ImportError as ie:
        print('Warning: usearch not installed or import failed. you might need to install libsqlite3-dev. '
              'For macs do `brew install libsqlite3-dev`. For linux `apt install libsqlite3-dev`. ', ie) if not q else None
    except subprocess.CalledProcessError as e:
        out = subprocess.run(['otool', '-l', dylib], capture_output=True, text=True).stdout
        if '/usr/lib' in out:
            if not q: print('rpath already present. Skip.')
            return
        print(f'✗ install_name_tool failed: {e} \n\n {e.stderr}')
    except Exception as e: print(f'Unexpected error during fix: {e}')

In [8]:
#| hide
usearch_fix()